In [1]:
import os
from pycocotools.coco import COCO
import csv, json
import pandas as pd
import cv2

In [31]:
# copy and rename files
import os
import shutil

def copy_rename(org_dir, new_dir, old_file_name, new_file_name):
    src_dir = ("../images/" + org_dir + "/")
    dst_dir = ("../images/" + new_dir + "/")
    src_file = os.path.join(src_dir, old_file_name)
    shutil.copy(src_file,dst_dir)
    dst_file = os.path.join(dst_dir, old_file_name)
    new_dst_file_name = os.path.join(dst_dir, new_file_name)
    os.rename(dst_file, new_dst_file_name)

In [17]:
def createAnnFile(output, visited, aug):
    df = pd.read_csv('../annotations/otherImgAnnotations.csv') 
    images = df["image"].tolist()
    caption = df["pair_caption"].tolist()
    for i, im in enumerate(images):
        im = im.split('.')[0]
        split_id = im.split("_")
        im_id = str(split_id[3]) + str(split_id[4]) + aug + ".jpg"
        if im_id not in visited:
            output["images"].append({"id": int(im_id.split(".")[0]), "file_name": im_id})
            visited[im_id] = True
        output["annotations"].append({"image_id": int(im_id.split(".")[0]), "caption": caption[i]})
    return visited, output

In [2]:
instancesFile = '../annotations/instances_val2017.json'
annFile = '../annotations/captions_val2017.json'
cocoInstances = COCO(instancesFile)
cocoAnn = COCO(annFile)

loading annotations into memory...
Done (t=0.83s)
creating index...
index created!
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [11]:
# create file called other img annotations
with open('../annotations/otherImgAnnotations.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['image', 'gt_caption'])
    for image in os.listdir('../images/other'):
        img_id = int(image.split('_')[3])
        annIds = cocoAnn.getAnnIds(imgIds = img_id)
        anns = cocoAnn.loadAnns(ids = annIds)
        for ann in anns:
            writer.writerow([image, ann])

In [6]:
df = pd.read_csv('../annotations/otherImgAnnotations.csv')
images = df["image"].tolist()
caption = df["pair_caption"].tolist()
visited = {}
output = {"annotations":[], "images":[]}
for i, im in enumerate(images):
    im_id = str(im.split("_")[3]) + str(im.split("_")[4])
    if im_id not in visited:
        output["images"].append({"id": int(im_id.split(".")[0]), "file_name": im_id})
        visited[im_id] = True
    output["annotations"].append({"image_id": int(im_id.split(".")[0]), "caption": caption[i]})

In [7]:
with open('../annotations/augAnn.json', 'w') as f:
    json.dump(output, f)

In [10]:
for file in os.listdir("../images/other images (resized)/data augmentations/flipped/flipped blocked/"):
    if file != '.DS_Store':
        filename = "../images/other images (resized)/data augmentations/flipped/flipped blocked/" + file
        print(filename)
        new_filename = "../images/other images (resized)/data augmentations/flipped/flipped blocked/" + file
        img = cv2.imread(filename)
        w, h, _ = img.shape

        scale_percent = 400 / w
        height = int(h * scale_percent)
        dsize = (height, 400)
        output = cv2.resize(img, dsize, interpolation = cv2.INTER_AREA)
        cv2.imwrite(new_filename, output) 

../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_handbag_153529_1_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_hairdrier_384136_3_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_refrigerator_456662_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_racket_55950_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_refrigerator_150417_3_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_sports_33759_1_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_frisbee_127263_1_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_frisbee_88485_3_flipped_blocked.png
../images/other images (resized)/data augmentations/flipp

../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_toothbrush_465179_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_refrigerator_22705_1_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_skateboard_128699_2_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_toothbrush_324614_2_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_skateboard_125472_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_sports_515982_4_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_surfboard_177357_2_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_skateboard_157365_1_flipped_blocked.png
../images/other images (resized)/data augmen

../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_tie_21604_4_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_toothbrush_507473_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_skateboard_31093_1_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_sports_135604_2_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_surfboard_54164_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_racket_85772_5_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_m_hairdrier_350002_3_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/flipped blocked/f_f_surfboard_54164_3_flipped_blocked.png
../images/other images (resized)/data augmentations/flipped/fli

In [39]:
visited = {}
output = {"annotations":[], "images":[]}
visited, output = createAnnFile(output, visited, "1")
visited, output = createAnnFile(output, visited, "")
visited, output = createAnnFile(output, visited, "2")
# visited, output = createAnnFile(output, visited, "3")

In [40]:
with open('../annotations/augAnn_12.json', 'w') as f:
    json.dump(output, f)

In [46]:
for im in os.listdir('../images/augmentation_resize/original_images'):
    print(im)
    org_dir = 'augmentation_resize/original_images'
    new_dir = 'aug12'
    old_file_name = im
    stripped = im.split('.')[0]
    new_file_name = stripped.split("_")[3] + stripped.split("_")[4] + ".jpg"
    copy_rename(org_dir, new_dir, old_file_name, new_file_name)

f_f_frisbee_127263_1.jpg
f_f_frisbee_127263_2.jpg
f_f_frisbee_127263_3.jpg
f_f_frisbee_127263_5.jpg
f_f_frisbee_227482_1.jpg
f_f_frisbee_227482_2.jpg
f_f_frisbee_227482_4.jpg
f_f_frisbee_227482_5.jpg
f_f_frisbee_291619_3.jpg
f_f_frisbee_291619_4.jpg
f_f_frisbee_291619_5.jpg
f_f_frisbee_325991_1.jpg
f_f_frisbee_325991_2.jpg
f_f_frisbee_325991_5.jpg
f_f_frisbee_328238_1.jpg
f_f_frisbee_328238_3.jpg
f_f_frisbee_328238_4.jpg
f_f_frisbee_328238_5.jpg
f_f_frisbee_88485_3.jpg
f_f_frisbee_88485_4.jpg
f_f_frisbee_88485_5.jpg
f_f_racket_127530_2.jpg
f_f_racket_127530_3.jpg
f_f_racket_127530_4.jpg
f_f_racket_127530_5.jpg
f_f_racket_19432_1.jpg
f_f_racket_19432_3.jpg
f_f_racket_19432_4.jpg
f_f_racket_19432_5.jpg
f_f_racket_55950_1.jpg
f_f_racket_55950_2.jpg
f_f_racket_55950_4.jpg
f_f_racket_55950_5.jpg
f_f_racket_64718_2.jpg
f_f_racket_64718_3.jpg
f_f_racket_64718_4.jpg
f_f_racket_64718_5.jpg
f_f_racket_85772_1.jpg
f_f_racket_85772_2.jpg
f_f_racket_85772_3.jpg
f_f_racket_85772_4.jpg
f_f_racket_857